In [1]:
#Include some software
import numpy as np
from pandas import DataFrame
from numpy.linalg import inv, norm
from scipy.linalg import svd  
from sympy import Matrix, latex, symbols, Symbol
import matplotlib.pyplot as plt


In [7]:

class Tomography:
    
    s_air = 1/330
    c_air = 330  #m/sec
    
    def __init__(self,angle,time):
        self.angle = angle
        self.time = time
        
        
    def __repr__(self):
        K_ang = ['(Angle_' + str(i+1) + ') ' + str(Angle[i]) for i in range(len(Angle))]
        V_tim = ['(time_' + str(i+1) + ') ' + str(Time_msec[i]) for i in range(len(Time_msec))]
        return f'{dict(zip(K_ang,V_tim))}'
    
    def data_frame(self):
        Time_sec = np.array(self.time) * 10**-3
        df = DataFrame([self.angle,np.array(self.angle)*(np.pi/180), self.time,Time_sec],
         index = ['Angle (deg)','Angle (rad)','Time (msec)', 'Time (sec)']).T
        return df
    
    def plot(self):
        plt.plot(self.time, self.angle,'.', color='black')
        plt.xlabel('$\mathbf{Time}$ (msec)')
        plt.ylabel('$\mathbf{Angle}$ (degrees)')
        plt.show()
    
    
    def Ray_points(self,x):
        Ly = lambda y,deg: (y,np.tan(deg*(np.pi/180))*y)
        Lx = lambda x,deg: (x/np.tan(deg*(np.pi/180)),x)
        X = [list(filter(lambda x: 1<=x[0]<=5 and 1<=x[1]<=5 ,
         [Lx(i,self.angle[j]) for i in range(1,5+1)])) for j in range(15)]
        Y = [list(filter(lambda x: 5 >= x[1] >=1 and 1<=x[0]<=5 ,
         [Ly(i,self.angle[j]) for i in range(1,5+1)] )) for j in range(15) ]
        Z = [X[i]+Y[i] for i in range(len(X))] 
        Ray = lambda i: sorted(Z[i])
        return Ray(x)
        
    def Ray_dist_in(self):
        
        Ly = lambda y,deg: (y,np.tan(deg*(np.pi/180))*y)
        Lx = lambda x,deg: (x/np.tan(deg*(np.pi/180)),x)
        
        
        X = [list(filter(lambda x: 1<=x[0]<=5 and 1<=x[1]<=5 ,
         [Lx(i,self.angle[j]) for i in range(1,5+1)])) for j in range(15)]
        
        Y = [list(filter(lambda x: 5 >= x[1] >=1 and 1<=x[0]<=5 ,
         [Ly(i,self.angle[j]) for i in range(1,5+1)] )) for j in range(15) ]
        
        Z = [X[i]+Y[i] for i in range(len(X))] # intersection points of the line_i with the squares
        
        Ray = lambda i: sorted(Z[i])
        
        point_dist = lambda P: np.sqrt((P[1][0] - P[0][0])**2 + (P[1][1] - P[0][1])) 
        #i.e P = ((0,0), (1,1)) --> sqrt(2)
        
        POINTS = lambda x : [(Ray(x)[i],Ray(x)[i+1]) for i in range(len(Ray(x))-1)] #pair points
        DIST_OF_ALL = [list(map(point_dist, POINTS(i))) for i in range(15)]     #all the dists of the pairs
        
        DOA = [DIST_OF_ALL[i][j] for i in range(15) for j in range(len(DIST_OF_ALL[i]))]
        
        counter_of_h_i = sum(list(map(len, DIST_OF_ALL)))
        
        L = dict(zip(['h_'+str(i) for i in range(1,counter_of_h_i+1)], DOA)) 
        
        H_val = list(L.values())[:29-1] + 4*[np.sqrt(2)] + list(L.values())[35:]
        
        H = dict(zip(['h_'+str(i+1) for i in range(len(H_val))], H_val))
        
        return H
    
    def Ray_dist_out(self, Dict=None):
        point_dist = lambda P: np.sqrt((P[1][0] - P[0][0])**2 + (P[1][1] - P[0][1])) 
        min_max = lambda points_arr: [min(points_arr)] + [max(points_arr)]  #keeps only the min,max points
        min_max_Dist = [min_max(self.Ray_points(i)) for i in range(15)]
        output = 8 - np.array(list(map(point_dist, min_max_Dist)))
        
        if Dict != None:
            return dict(zip(['h_'+str(i+1)+'^out' for i in range(15)], output)) #as a dict
        
        return output #dist of the ray out of the testing blocks
    
    def time_out(self):
        return dict(zip(['t_'+str(i+1)+'^out' for i in range(15)], self.Ray_dist_out()/330))

    def time_in_minus_out(self, Dict=None):
        output = np.array(self.time) - np.array(list(self.time_out().values()))
        if Dict != None:
            return dict(zip(['t\'_'+str(i+1) for i in range(15)], output))
        return output
    
    def data_frame_all(self):
        Time_sec = np.array(self.time) * 10**-3
        df = DataFrame([self.angle,np.array(self.angle)*(np.pi/180), self.time,
                        Time_sec, self.Ray_dist_out(), list(self.time_out().values()),self.time_in_minus_out()],
         index = ['Angle (deg)','Angle (rad)','Time (msec)', 'Time (sec)',
                  '$h^{out}$','$t^{out}$','t \'']).T
        return df
        
        
Angle = [
    12,15,18,23,32,36,42,45,50,55,60,65,70,75,78
]
Time_msec = [
    22.42,20.59,18.628,16.214,13.112,11.695,9.62,8.358,10.335,12.006,13.822,15.48,17.189,20.58,23.22
]

Tom_test = Tomography(Angle,Time_msec)

d= np.array(list(map(lambda t: t[0],np.array(Tom_test.data_frame_all()[['t \'']]))))
d

array([22.39693131, 20.56998639, 18.61005587, 16.19838936, 13.09896645,
       11.6827888 ,  9.60883689,  8.3473095 , 10.32210381, 11.991257  ,
       13.80567576, 15.46227695, 17.16993991, 20.55932173, 23.19741544])

In [32]:
def format_matrix(matrix, demical=None, latex=None): #format(. ,"n.f") but for matrices
    if (demical==None): demical= "."+str(5)+"f"
    else:
        demical = "."+str(demical)+"f"
    f_oper = lambda arr: np.array([float(format(arr[i],demical)) for i in range(len(arr))])    
    if latex==True:
        return Matrix(np.array(list(map(f_oper, matrix))))
    return np.array(list(map(f_oper, matrix)))

fm = lambda matrix, demical=None, latex=None: format_matrix(matrix, demical, latex) 

In [33]:
h = list(Tom_test.Ray_dist_in().values())

G = np.array([
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,h[1-1]],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,h[2-1],h[3-1]],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,h[4-1],h[5-1]],
    [0,0,0,0,0,0,0,0,0,0,0,h[9-1],0,h[6-1],h[7-1],h[8-1]],
    [0,0,0,0,0,0,0,h[15-1],0,0,h[13-1],h[14-1],h[10-1],h[11-1],h[12-1],0],
    [0,0,0,0,0,0,0,h[21-1],0,h[18-1],h[19-1],h[20-1],h[16-1],h[17-1],0,0],
    [0,0,0,h[28-1],0,0,h[26-1],h[27-1],0,h[24-1],h[25-1],0,h[22-1],h[23-1],0,0],
    [0,0,0,h[32-1],0,0,h[31-1],0,0,h[30-1],0,0,h[29-1],0,0,0],
    [0,0,h[38-1],h[39-1],0,h[36-1],h[37-1],0,h[34-1],h[35-1],0,0,h[33-1],0,0,0],
    [0,h[44-1],h[45-1],0,0,h[43-1],0,0,h[41-1],h[42-1],0,0,h[40-1],0,0,0],
    [0,h[50-1],0,0,h[48-1],h[49-1],0,0,h[47-1],0,0,0,h[46-1],0,0,0],
    [h[53-1],h[54-1],0,0,h[52-1],0,0,0,h[51-1],0,0,0,0,0,0,0],
    [h[57-1],0,0,0,h[56-1],0,0,0,h[55-1],0,0,0,0,0,0,0],
    [h[59-1],0,0,0,h[58-1],0,0,0,0,0,0,0,0,0,0,0],
    [h[60-1],0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
])
Matrix(format_matrix(G,3))

Matrix([
[    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0, 0.387],
[    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0, 0.379, 1.126],
[    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0, 1.073, 1.151],
[    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0, 0.453,     0,  0.83, 1.194, 0.899],
[    0,     0,     0,     0,     0,     0,     0, 0.405,     0,     0, 1.067, 1.069,  0.64, 1.275, 0.407,     0],
[    0,     0,     0,     0,     0,     0,     0, 1.179,     0, 0.491, 1.314, 0.332, 0.918, 1.055,     0,     0],
[    0,     0,     0, 0.902,     0,     0, 1.024, 0.771,     0, 1.144, 0.639,     0, 1.262, 0.498,     0,     0],
[    0,     0,     0, 1.414,     0,     0, 1.414,     0,     0, 1.414,     0,     0, 1.414,     0,     0,     0],
[    0,     0, 1.087, 0.521,     0, 0.899, 0.743,     0, 0.698,  0.94,     0,  

In [95]:

#SVD (of the kernel matrix G):
N = len(Angle)
M = 16  #model parameters

U, S, V = svd(G) #V--> V.T
f = lambda i,j: S[i] if i==j else 0    
Sigma = np.array(Matrix(N, M, f), dtype=float)
Sigma_p = fm(np.diag(S),10)[:-1,:-1]
p = len(fm(np.diag(S),10)[:-1,:-1])
U_p = U[:,:p]
V_p = V[:p,:]

In [154]:
print(f'G shape: {G.shape}')
print(f'Sigma shape: {Sigma.shape}')
print(f'Sigma_p shape: {Sigma_p.shape}')
print(f'U shape: {U.shape}')
print(f'V shape: {V.T.shape}')
print(f'U_p shape: {U_p.shape}')
print(f'V_p shape: {V_p.T.shape}')
print(f'singular values > epsilon=0.001: {p}')

G shape: (15, 16)
Sigma shape: (15, 16)
Sigma_p shape: (14, 14)
U shape: (15, 15)
V shape: (16, 16)
U_p shape: (15, 14)
V_p shape: (16, 14)
singular values > epsilon=0.001: 14


In [150]:
Matrix(fm(Sigma_p ,3))

Matrix([
[4.561,     0,     0,     0,     0,     0,     0,     0,   0,     0,     0,     0,     0,     0],
[    0, 3.244,     0,     0,     0,     0,     0,     0,   0,     0,     0,     0,     0,     0],
[    0,     0, 2.874,     0,     0,     0,     0,     0,   0,     0,     0,     0,     0,     0],
[    0,     0,     0, 2.346,     0,     0,     0,     0,   0,     0,     0,     0,     0,     0],
[    0,     0,     0,     0, 2.122,     0,     0,     0,   0,     0,     0,     0,     0,     0],
[    0,     0,     0,     0,     0, 1.328,     0,     0,   0,     0,     0,     0,     0,     0],
[    0,     0,     0,     0,     0,     0, 1.052,     0,   0,     0,     0,     0,     0,     0],
[    0,     0,     0,     0,     0,     0,     0, 0.528,   0,     0,     0,     0,     0,     0],
[    0,     0,     0,     0,     0,     0,     0,     0, 0.5,     0,     0,     0,     0,     0],
[    0,     0,     0,     0,     0,     0,     0,     0,   0, 0.337,     0,     0,     0,     0],
[    0,    

In [155]:
#Svd matrix
Matrix(format_matrix(U @ Sigma @ V, 7) ) == Matrix(format_matrix(G,7))

True

In [165]:
Matrix(format_matrix(U,2)) # ~ matrix U

Matrix([
[    0, -0.01,  0.04,  0.11, -0.02,  0.01,  0.16, -0.18,  0.31,  0.17, -0.22, -0.01, -0.01,  0.02,  0.87],
[-0.01, -0.05,  0.16,   0.4, -0.08,  0.03,  0.37, -0.32,  0.51,  0.19,  -0.2, -0.03, -0.03,  0.04, -0.47],
[-0.01, -0.08,  0.25,  0.56,  -0.1,  0.03,  0.24,  0.04, -0.17, -0.38,  0.59, -0.03, -0.01,     0,  0.17],
[-0.06, -0.18,   0.4,  0.46, -0.05, -0.02, -0.35,  0.27, -0.34,  0.23, -0.46,  0.08,  0.05, -0.05,     0],
[-0.23, -0.32,  0.47, -0.21,  0.13, -0.07, -0.52, -0.26,  0.33, -0.09,  0.25, -0.13,  -0.1,  0.12,     0],
[-0.33, -0.33,  0.34, -0.37,  0.11, -0.01,   0.5,  0.11, -0.09,     0, -0.04,   0.3,  0.26, -0.31,     0],
[-0.49, -0.23, -0.12, -0.08, -0.19,  0.14,  0.24,  0.14,  -0.2,  0.07, -0.11, -0.43, -0.37,  0.42,     0],
[-0.52,  -0.1, -0.43,  0.17, -0.34,  0.19, -0.26, -0.22,  0.05, -0.02,  0.07,  0.43,  0.19,  -0.1,     0],
[-0.44,  0.21, -0.17,  0.18,  0.24, -0.42, -0.05,   0.3,  0.24, -0.05,     0, -0.38,   0.1, -0.41,     0],
[ -0.3,  0.41,  0.09,   0.1,

In [163]:
Matrix(format_matrix(V,2)) # ~ matrix U

Matrix([
[-0.02,  -0.1, -0.17, -0.31, -0.06,  -0.2, -0.34, -0.19, -0.21, -0.43, -0.22, -0.08, -0.58,  -0.2, -0.04, -0.02],
[  0.2,  0.35,   0.2, -0.07,  0.35,  0.33, -0.07, -0.21,  0.52, -0.06, -0.28, -0.16, -0.03, -0.31, -0.14,  -0.1],
[ 0.17,   0.2, -0.03, -0.28,  0.26,  0.05,  -0.3,  0.17,  0.22, -0.24,   0.3,  0.28, -0.05,  0.43,  0.35,  0.29],
[-0.14, -0.04,  0.12,  0.11, -0.15,   0.1,  0.13, -0.25,     0,  0.08, -0.32, -0.06, -0.03, -0.14,  0.52,  0.66],
[-0.55,  0.04,  0.34, -0.25, -0.47,  0.41, -0.24,  0.02,  0.11, -0.11,  0.07,  0.07,  0.09,  0.06, -0.07, -0.12],
[-0.58,  0.55, -0.52,  0.14,  0.09, -0.19,  0.08,  0.05,  0.12, -0.04,     0, -0.07,     0, -0.04, -0.01,  0.04],
[ 0.05, -0.02,  0.01, -0.18, -0.02,  0.02, -0.16,  0.53, -0.02,  0.11,  0.23, -0.52,  0.03,  -0.3, -0.22,  0.42],
[-0.34, -0.24,  0.35, -0.07,  0.31, -0.31,  0.09,  0.24,  0.26,  0.22, -0.09, -0.22, -0.33,  0.15,  0.28, -0.24],
[-0.26, -0.24,  0.22,  0.03,  0.27, -0.23,  0.09, -0.25,  0.15, -0.08,  0.22,  

In [103]:
Matrix(format_matrix(Sigma,4)) # ~ matrix Sigma

Matrix([
[4.5607,      0,      0,     0,     0,      0,      0,      0,      0,     0,      0,      0,      0,      0, 0, 0],
[     0, 3.2436,      0,     0,     0,      0,      0,      0,      0,     0,      0,      0,      0,      0, 0, 0],
[     0,      0, 2.8743,     0,     0,      0,      0,      0,      0,     0,      0,      0,      0,      0, 0, 0],
[     0,      0,      0, 2.346,     0,      0,      0,      0,      0,     0,      0,      0,      0,      0, 0, 0],
[     0,      0,      0,     0, 2.122,      0,      0,      0,      0,     0,      0,      0,      0,      0, 0, 0],
[     0,      0,      0,     0,     0, 1.3278,      0,      0,      0,     0,      0,      0,      0,      0, 0, 0],
[     0,      0,      0,     0,     0,      0, 1.0515,      0,      0,     0,      0,      0,      0,      0, 0, 0],
[     0,      0,      0,     0,     0,      0,      0, 0.5284,      0,     0,      0,      0,      0,      0, 0, 0],
[     0,      0,      0,     0,     0,      0,      0, 

In [104]:
Matrix(format_matrix(U_p,2))

Matrix([
[    0, -0.01,  0.04,  0.11, -0.02,  0.01,  0.16, -0.18,  0.31,  0.17, -0.22, -0.01, -0.01,  0.02],
[-0.01, -0.05,  0.16,   0.4, -0.08,  0.03,  0.37, -0.32,  0.51,  0.19,  -0.2, -0.03, -0.03,  0.04],
[-0.01, -0.08,  0.25,  0.56,  -0.1,  0.03,  0.24,  0.04, -0.17, -0.38,  0.59, -0.03, -0.01,     0],
[-0.06, -0.18,   0.4,  0.46, -0.05, -0.02, -0.35,  0.27, -0.34,  0.23, -0.46,  0.08,  0.05, -0.05],
[-0.23, -0.32,  0.47, -0.21,  0.13, -0.07, -0.52, -0.26,  0.33, -0.09,  0.25, -0.13,  -0.1,  0.12],
[-0.33, -0.33,  0.34, -0.37,  0.11, -0.01,   0.5,  0.11, -0.09,     0, -0.04,   0.3,  0.26, -0.31],
[-0.49, -0.23, -0.12, -0.08, -0.19,  0.14,  0.24,  0.14,  -0.2,  0.07, -0.11, -0.43, -0.37,  0.42],
[-0.52,  -0.1, -0.43,  0.17, -0.34,  0.19, -0.26, -0.22,  0.05, -0.02,  0.07,  0.43,  0.19,  -0.1],
[-0.44,  0.21, -0.17,  0.18,  0.24, -0.42, -0.05,   0.3,  0.24, -0.05,     0, -0.38,   0.1, -0.41],
[ -0.3,  0.41,  0.09,   0.1,  0.48, -0.24,  0.07, -0.14, -0.16,  0.15,  0.07,  0.38, -0.07,

In [111]:
Matrix(format_matrix(U,2))==Matrix(format_matrix(U_p,2))

False

In [112]:
Matrix(format_matrix(Sigma_p,4))

Matrix([
[4.5607,      0,      0,     0,     0,      0,      0,      0,      0,     0,      0,      0,      0,      0],
[     0, 3.2436,      0,     0,     0,      0,      0,      0,      0,     0,      0,      0,      0,      0],
[     0,      0, 2.8743,     0,     0,      0,      0,      0,      0,     0,      0,      0,      0,      0],
[     0,      0,      0, 2.346,     0,      0,      0,      0,      0,     0,      0,      0,      0,      0],
[     0,      0,      0,     0, 2.122,      0,      0,      0,      0,     0,      0,      0,      0,      0],
[     0,      0,      0,     0,     0, 1.3278,      0,      0,      0,     0,      0,      0,      0,      0],
[     0,      0,      0,     0,     0,      0, 1.0515,      0,      0,     0,      0,      0,      0,      0],
[     0,      0,      0,     0,     0,      0,      0, 0.5284,      0,     0,      0,      0,      0,      0],
[     0,      0,      0,     0,     0,      0,      0,      0, 0.4997,     0,      0,      0,      0,  

In [109]:
G_g = V_p.T @ inv(Sigma_p) @ U_p.T
m_est = G_g @ d
SVD_d_pre = G @ m_est

In [172]:
k = norm(fm(G,4),'fro')*norm(fm(G_g,4),'fro')
k

211.92805200441316

In [222]:
Matrix(list(map(lambda x: float(format(x,".3f")),d))) #d^obs

Matrix([
[22.397],
[ 20.57],
[ 18.61],
[16.198],
[13.099],
[11.683],
[ 9.609],
[ 8.347],
[10.322],
[11.991],
[13.806],
[15.462],
[ 17.17],
[20.559],
[23.197]])

In [189]:
G.shape

(15, 16)

In [190]:
G_g.shape

(16, 15)

In [211]:
Matrix(m_est)

Matrix([
[ 42.4012048821295],
[ 2.92160154066372],
[-45.9423193414243],
[-125.457065331039],
[-44.6565443873581],
[-15.5055809044644],
[ 73.8204067855601],
[-53.2033679958193],
[ 38.3090283029193],
[ 36.7889245336199],
[ 17.5138894950711],
[-12.3891054042949],
[ 20.7496630410947],
[ 17.4826807501651],
[-15.6416962271629],
[ 28.8900710148507]])

In [117]:
Matrix(SVD_d_pre)

Matrix([
[11.1900528034368],
[26.6039354898852],
[16.4775844513155],
[16.1991536299383],
[13.1007117174831],
[11.6833944021517],
[9.60890689007354],
[8.34658807730831],
[10.3212418385544],
[11.9908211163506],
[13.8053362681345],
[15.4625654140672],
[17.1701762512642],
[20.5597375062357],
[23.1974341934344]])

In [128]:
Matrix(d)

Matrix([
[22.3969313094913],
[20.5699863862922],
[18.6100558687503],
[16.1983893612493],
[13.0989664500106],
[11.6827888025851],
[9.60883688993777],
[8.34730950289394],
[10.3221038068206],
[11.9912570037648],
[13.8056757638578],
[15.4622769452867],
[ 17.169939906517],
[20.5593217281002],
[23.1974154356068]])

In [130]:
norm(d - SVD_d_pre, 2) #l2 norm

12.905429394758873

In [131]:
norm(d - SVD_d_pre, np.inf) #linf norm

11.206878506054517

In [132]:
np.mean(1/m_est)

0.0218582431548799

In [134]:
S2 = (1/(15-1)) * ( sum(d**2) - (1/15)*sum(d)**2 )
S2

22.803359750086134

In [133]:
s2 = (1/(15-1)) * ( sum(SVD_d_pre**2) - (1/15)*sum(SVD_d_pre)**2 )
s2

26.864160632766048

In [246]:
par_analysis_matrix = G_g @ G # = V_p.T@V_p
Matrix(fm(par_analysis_matrix,3))

Matrix([
[1.0,   0,      0,      0,   0,      0,      0,      0,   0,      0,      0,      0,      0,      0,   0,   0],
[  0, 1.0,      0,      0,   0,      0,      0,      0,   0,      0,      0,      0,      0,      0,   0,   0],
[  0,   0,  0.955,  0.001,   0,  0.077,  -0.01,  0.018,   0,  0.121,  0.032,  0.082, -0.113, -0.044,   0,   0],
[  0,   0,  0.001,  0.921,   0, -0.051,  0.011,  0.135,   0, -0.007, -0.144,  0.139,  0.075, -0.076,   0,   0],
[  0,   0,      0,      0, 1.0,      0,      0,      0,   0,      0,      0,      0,      0,      0,   0,   0],
[  0,   0,  0.077, -0.051,   0,   0.84,  0.023,  0.054,   0, -0.207, -0.144, -0.049,  0.235,  0.027,   0,   0],
[  0,   0,  -0.01,  0.011,   0,  0.023,  0.996, -0.015,   0,  0.026,  0.027, -0.002, -0.034,  0.001,   0,   0],
[  0,   0,  0.018,  0.135,   0,  0.054, -0.015,  0.762,   0,  -0.04,  0.234, -0.274, -0.079,   0.15,   0,   0],
[  0,   0,      0,      0,   0,      0,      0,      0, 1.0,      0,      0,      0,      0,   

In [228]:
data_analysis_matrix = U_p @ U_p.T # = G @ G_g
Matrix(fm(data_analysis_matrix,3)) 

Matrix([
[ 0.246, 0.406, -0.143,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
[ 0.406, 0.781,  0.077,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
[-0.143, 0.077,  0.973,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
[     0,     0,      0, 1.0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
[     0,     0,      0,   0, 1.0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
[     0,     0,      0,   0,   0, 1.0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
[     0,     0,      0,   0,   0,   0, 1.0,   0,   0,   0,   0,   0,   0,   0,   0],
[     0,     0,      0,   0,   0,   0,   0, 1.0,   0,   0,   0,   0,   0,   0,   0],
[     0,     0,      0,   0,   0,   0,   0,   0, 1.0,   0,   0,   0,   0,   0,   0],
[     0,     0,      0,   0,   0,   0,   0,   0,   0, 1.0,   0,   0,   0,   0,   0],
[     0,     0,      0,   0,   0,   0,   0,   0,   0,   0, 1.0,   0,   0,   0,   0],
[     0,     0,      0,   0,   0,   0,   0,   0,   0,   

In [201]:
Matrix(np.diag(fm(par_analysis_matrix,3)))

Matrix([
[  1.0],
[  1.0],
[0.955],
[0.921],
[  1.0],
[ 0.84],
[0.996],
[0.762],
[  1.0],
[0.676],
[0.714],
[0.602],
[0.654],
[0.881],
[  1.0],
[  1.0]])

In [202]:
Covd = S2*np.eye(len(d))
Matrix(fm(Covd,3))

Matrix([
[22.803,      0,      0,      0,      0,      0,      0,      0,      0,      0,      0,      0,      0,      0,      0],
[     0, 22.803,      0,      0,      0,      0,      0,      0,      0,      0,      0,      0,      0,      0,      0],
[     0,      0, 22.803,      0,      0,      0,      0,      0,      0,      0,      0,      0,      0,      0,      0],
[     0,      0,      0, 22.803,      0,      0,      0,      0,      0,      0,      0,      0,      0,      0,      0],
[     0,      0,      0,      0, 22.803,      0,      0,      0,      0,      0,      0,      0,      0,      0,      0],
[     0,      0,      0,      0,      0, 22.803,      0,      0,      0,      0,      0,      0,      0,      0,      0],
[     0,      0,      0,      0,      0,      0, 22.803,      0,      0,      0,      0,      0,      0,      0,      0],
[     0,      0,      0,      0,      0,      0,      0, 22.803,      0,      0,      0,      0,      0,      0,      0],
[     0,      0

In [206]:
Covm = G_g @ Covd @ G_g.T
Matrix(fm(Covm,2))

Matrix([
[  76.19,   -35.9,   -157.7,  -375.23, -150.93,  -70.41,   207.88,  -154.17,   155.7,   115.31,   47.97,  -59.26,    52.04,   32.38,      0,      0],
[  -35.9,   253.4,    268.9,   226.24,  162.82,   44.67,  -126.95,    95.17, -361.53,   -44.08,  -20.89,   51.25,   -55.21,  -28.01,      0,      0],
[ -157.7,   268.9,   939.74,   2577.8,  416.68,  382.97, -1066.35,  1157.27, -584.36,  -984.93, -231.07,  425.52,  -540.42, -245.68,  -2.28,   4.53],
[-375.23,  226.24,   2577.8,  9475.88,  862.33, 1379.02, -3854.34,  4245.99,  -934.2, -3749.17, -906.01, 1488.22, -1897.68, -830.77, -27.06,  24.54],
[-150.93,  162.82,   416.68,   862.33,  382.49,  192.53,  -478.39,   357.61, -482.38,  -265.46, -113.25,  139.28,  -118.48,  -76.11,      0,      0],
[ -70.41,   44.67,   382.97,  1379.02,  192.53,   337.1,  -425.35,   683.09, -240.01,  -652.29, -141.56,  209.08,  -317.92, -103.26, -13.15,   7.82],
[ 207.88, -126.95, -1066.35, -3854.34, -478.39, -425.35,  2079.01, -1548.19,  519.64,  1201

In [262]:
Matrix(fm(Covm,2))

Matrix([
[  76.19,   -35.9,   -157.7,  -375.23, -150.93,  -70.41,   207.88,  -154.17,   155.7,   115.31,   47.97,  -59.26,    52.04,   32.38,      0,      0],
[  -35.9,   253.4,    268.9,   226.24,  162.82,   44.67,  -126.95,    95.17, -361.53,   -44.08,  -20.89,   51.25,   -55.21,  -28.01,      0,      0],
[ -157.7,   268.9,   939.74,   2577.8,  416.68,  382.97, -1066.35,  1157.27, -584.36,  -984.93, -231.07,  425.52,  -540.42, -245.68,  -2.28,   4.53],
[-375.23,  226.24,   2577.8,  9475.88,  862.33, 1379.02, -3854.34,  4245.99,  -934.2, -3749.17, -906.01, 1488.22, -1897.68, -830.77, -27.06,  24.54],
[-150.93,  162.82,   416.68,   862.33,  382.49,  192.53,  -478.39,   357.61, -482.38,  -265.46, -113.25,  139.28,  -118.48,  -76.11,      0,      0],
[ -70.41,   44.67,   382.97,  1379.02,  192.53,   337.1,  -425.35,   683.09, -240.01,  -652.29, -141.56,  209.08,  -317.92, -103.26, -13.15,   7.82],
[ 207.88, -126.95, -1066.35, -3854.34, -478.39, -425.35,  2079.01, -1548.19,  519.64,  1201

In [248]:
size = data_analysis_matrix.shape[0]
spread = norm(data_analysis_matrix - np.eye(size), 'fro')**2
spread

1.0000000032739296

In [249]:
size = par_analysis_matrix.shape[0]
spread = norm(par_analysis_matrix - np.eye(size), 'fro')**2
spread

2.000000629250338

In [250]:
par_analysis_matrix.shape

(16, 16)

In [252]:
data_analysis_matrix.shape

(15, 15)

## Maris Ioannis 2022